In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests


car_make = []
car_model = []
kilometers = []
fuel_type = []
transmission = []
price = []
location = []


url = 'https://www.cars24.com/buy-used-maruti-cars-mumbai/?sort=bestmatch&serveWarrantyCount=true&listingSource=TabFilter&storeCityId=2378'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0 Safari/537.36'
}

response = requests.get(url, headers=headers)

if response.status_code != 200:
    print("Failed to retrieve page:", response.status_code)
    exit()

soup = BeautifulSoup(response.text, 'html.parser')

car_cards = soup.find_all('a', {'class': 'styles_carCardWrapper__sXLIp'})[:9]

for card in car_cards:
    
    title_span = card.find('span', {'class': 'sc-braxZu kjFjan'})
    if title_span:
        full_title = title_span.get_text().strip()
        parts = full_title.split()
        if len(parts) >= 3:
            car_make.append(parts[1])
            car_model.append(' '.join(parts[2:]))
        else:
            car_make.append('N/A')
            car_model.append('N/A')
    else:
        car_make.append('N/A')
        car_model.append('N/A')

    specs = card.find_all('p', {'class': 'sc-braxZu kvfdZL'})
    kilometers.append(specs[0].get_text().strip() if len(specs) > 0 else 'N/A')
    fuel_type.append(specs[1].get_text().strip() if len(specs) > 1 else 'N/A')
    transmission.append(specs[2].get_text().strip() if len(specs) > 2 else 'N/A')

   
    price_p = card.find('p', {'class': 'sc-braxZu cyPhJl'})
    price.append(price_p.get_text().strip() if price_p else 'N/A')

    
    loc_p = card.find('p', {'class': 'sc-braxZu dLJHhZ'}) 
    if not loc_p:
        loc_p = card.find('span', {'class': 'styles_cityName__location'})  
    if not loc_p:
        
        found_location = None
        for tag in card.find_all(['p', 'span']):
            text = tag.get_text().strip()
            if 'Mumbai' in text or 'Location' in text or 'City' in text:
                found_location = text
                break
        location.append(found_location if found_location else 'N/A')
    else:
        location.append(loc_p.get_text().strip())


df = pd.DataFrame({
    ' Car Make': car_make,
    'Model': car_model,
    'Kilometers Driven': kilometers,
    'Fuel Type': fuel_type,
    'Transmission Type': transmission,
    'Price': price,
    'Location': location
})

df.to_csv('cars24_maruti_mumbai_9.csv', index=False)
print("CSV saved as cars24_maruti_mumbai_9.csv")


CSV saved as cars24_maruti_mumbai_9.csv
